Sentiment Analysis of Multilingual(Tanglish) Movie Reviews

Using Tokenizer for word embedding



In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

data = pd.read_csv('/content/Tamil_sentiments.csv', encoding='utf-8')

In [43]:
data['text'].tail(5)

15739    lings \t ivaru cinemala laam nalla tha prasura...
15740    \t Pattaya Kilaputhupaa trailer... !!!!! Get R...
15741    lings \t En innum trending la varala? Ennada p...
15742        \t Rajnikant sir plz aap india ke pm ban jaao
15743    lings \t Enagada YouTube inum trending la add ...
Name: text, dtype: object

In [40]:
num_categories = 5

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['category'], test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)


y_train_cat = to_categorical(y_train, num_classes=num_categories)
y_test_cat = to_categorical(y_test, num_classes=num_categories)

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=max_len))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=num_categories, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train_pad, y_train_cat, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test_cat), callbacks=[early_stopping])
model.save('tanglish_sentiment_lstm.h5')
loss, accuracy = model.evaluate(X_test_pad, y_test_cat, verbose=0)
print('Accuracy:', accuracy)

Epoch 1/10
394/394 [==============================] - 186s 451ms/step - loss: 0.7923 - accuracy: 0.7350 - val_loss: 0.4733 - val_accuracy: 0.8250
Epoch 2/10
394/394 [==============================] - 166s 421ms/step - loss: 0.3612 - accuracy: 0.8638 - val_loss: 0.4190 - val_accuracy: 0.8269
Epoch 3/10
394/394 [==============================] - 166s 422ms/step - loss: 0.2981 - accuracy: 0.8888 - val_loss: 0.4135 - val_accuracy: 0.8453
Epoch 4/10
394/394 [==============================] - 168s 427ms/step - loss: 0.2553 - accuracy: 0.9073 - val_loss: 0.4283 - val_accuracy: 0.8399
Epoch 5/10
394/394 [==============================] - 164s 416ms/step - loss: 0.2187 - accuracy: 0.9205 - val_loss: 0.4562 - val_accuracy: 0.8358
Epoch 6/10
394/394 [==============================] - 166s 419ms/step - loss: 0.1931 - accuracy: 0.9298 - val_loss: 0.4784 - val_accuracy: 0.8323
Accuracy: 0.8323277235031128


In [41]:
from keras.models import load_model

model = load_model('tanglish_sentiment_lstm.h5')

test_text = ['padam nalla iruke']
test_seq = tokenizer.texts_to_sequences(test_text)
test_pad = pad_sequences(test_seq, maxlen=max_len)
pred = model.predict(test_pad)

for p in pred:
    print(np.argmax(p))

1/1 [==============================] - 0s 324ms/step
2


In [42]:
predicted_category = encoder.inverse_transform([np.argmax(pred)])
print(predicted_category)

['Positive']


Using CountVectorizer for word embedding

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils  import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

data = pd.read_csv('/content/Tamil_sentiments.csv', encoding='utf-8')

num_categories = 5

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['category'], test_size=0.2, random_state=42)

vectorizer = CountVectorizer(max_features=5000)

X_train_seq = vectorizer.fit_transform(X_train).toarray()
X_test_seq = vectorizer.transform(X_test).toarray()

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

y_train_cat = to_categorical(y_train, num_classes=num_categories)
y_test_cat = to_categorical(y_test, num_classes=num_categories)

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=max_len))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=num_categories, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train_pad, y_train_cat, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test_cat), callbacks=[early_stopping])

model.save('tanglish_sentiment_lstm_countvectorizer.h5')

loss, accuracy = model.evaluate(X_test_pad, y_test_cat, verbose=0)
print('Accuracy:', accuracy)

Epoch 1/10
394/394 [==============================] - 178s 445ms/step - loss: 1.0562 - accuracy: 0.6731 - val_loss: 1.0839 - val_accuracy: 0.6548
Epoch 2/10
394/394 [==============================] - 175s 445ms/step - loss: 1.0428 - accuracy: 0.6746 - val_loss: 1.0815 - val_accuracy: 0.6548
Epoch 3/10
394/394 [==============================] - 174s 441ms/step - loss: 1.0417 - accuracy: 0.6746 - val_loss: 1.0806 - val_accuracy: 0.6548
Epoch 4/10
394/394 [==============================] - 174s 441ms/step - loss: 1.0408 - accuracy: 0.6746 - val_loss: 1.0845 - val_accuracy: 0.6548
Epoch 5/10
394/394 [==============================] - 173s 438ms/step - loss: 1.0405 - accuracy: 0.6746 - val_loss: 1.0805 - val_accuracy: 0.6548
Epoch 6/10
394/394 [==============================] - 167s 424ms/step - loss: 1.0396 - accuracy: 0.6746 - val_loss: 1.0812 - val_accuracy: 0.6545
Epoch 7/10
394/394 [==============================] - 167s 424ms/step - loss: 1.0394 - accuracy: 0.6748 - val_loss: 1.0827 -

In [ ]:
from keras.models import load_model

model = load_model('/content/tanglish_sentiment_lstm_countvectorizer.h5')

test_text = [' நான் இன்னும் கூட உங்கள் புதிய படத்தை பார்க்க மிகவும் ஆசையாக இருக்கிறேன்']
test_seq = tokenizer.texts_to_sequences(test_text)
test_pad = pad_sequences(test_seq, maxlen=max_len)
pred = model.predict(test_pad)

for p in pred:
    print(np.argmax(p))

1/1 [==============================] - 0s 244ms/step
2


In [ ]:
predicted_category = encoder.inverse_transform([np.argmax(pred)])
print(predicted_category)

['Positive']
